In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
filename = "sap_storing_data_hu_project.csv" 
n = sum(1 for line in open(filename, encoding='utf8'))-1  
s = n//10
skip = sorted(random.sample(range(1, n+1), n-s))
df_not_cleaned = pd.read_csv(filename, skiprows=skip, low_memory=False)
#verwijder alle kolommen die niet van gebruik zijn:
df = df_not_cleaned.drop(columns=['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld', 'stm_geo_mld_uit_functiepl', 'stm_scenario_mon', 'stm_status_melding_sap', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_aanntpl_ddt', 'stm_geo_gst_uit_functiepl', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt', 'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz', 'stm_veroorz_groep', 'stm_veroorz_code', 'stm_progfh_gw_duur', 'stm_progfh_gw_lwd_tijd', 'stm_progfh_gw_lwd_datum', 'stm_progfh_gw_datum', 'stm_arbeid', 'stm_functiepl_gst', 'stm_functiepl_mld', 'stm_mon_begintijd', 'stm_mon_begindatum', 'stm_aangelegd_tijd', 'stm_aangelegd_dd', 'stm_dir_betrok_tr', 'stm_evb', 'stm_mon_vhdsincident', 'stm_mon_eind_ddt', 'stm_afspr_aanvangddt', 'stm_effect', 'stm_veroorz_tekst_kort', 'stm_projnr', 'stm_wbi_nummer', 'stm_status_gebr', 'stm_standplaats', 'stm_actie', 'stm_hinderwaarde', 'stm_rec_toegev_ddt', 'stm_afsluit_tijd', 'stm_afsluit_dd', 'stm_afsluit_ddt', 'stm_akkoord_melding_jn', 'stm_fact_jn', 'stm_status_sapnaarmon', 'stm_akkoord_mon_toewijz', 'stm_controle_dd', 'stm_mon_eind_tijd', 'stm_mon_eind_datum', 'stm_reactie_duur', 'stm_afspr_aanvangtijd', 'stm_afspr_aanvangdd', 'stm_historie_toelichting', 'stm_schade_verhaalb_jn', 'stm_x_bron_arch_dt', 'stm_x_bron_bestandsnaam', 'stm_x_bron_publ_dt', 'stm_rapportage_jaar', 'stm_rapportage_maand', 'stm_sorteerveld', 'stm_afspr_func_hersteltijd', 'stm_afspr_func_hersteldd', 'stm_relatiervo', 'stm_relatiervo_volgend', 'stm_relatiervo_vorig', 'stm_schade_statusdatum', 'stm_schade_status_ga', 'stm_schadenr', 'stm_x_start_sessie_dt', 'stm_x_bk', 'stm_x_run_id', 'stm_x_actueel_ind', 'stm_x_vervallen_ind'])
print("Done")

Done


# Data cleaning:


De te bepalen welke variabelen het meest van toepassing zijn om te gebruiken kijken we naar welke
kolommen informatie geven over de tijd van de melding, monteur. Tijd wanneer de melding word opgelost.
Wat voor melding het is en info over de melding/storing, en de locatie. Zo zijn de volgende kolommen
het meest van toepassing:

Over soort melding
- stm_sap_meldtekst (melding omschrijving)
- stm_geo_mld (melding code) - nodig
- stm_oorz_code (oorzaak code) - nodig
- stm_prioriteit (prioriteit)

Over tijd:
- stm_sap_melddatum (datump melding) - nodig
- stm_sap_meldtijd (tijdstip melding) - nodig
- stm_aanngeb_dd (datum aannemer gebeld)
- stm_aanngeb_tijd (tijd aannemer gebeld)
- stm_aanntpl_dd (datum aannemer op plek)
- stm_aanntpl_tijd (tijd aannemer op plek)

- stm_fh_dd (Datum van functie herstel) - nodig
- stm_fh_tijd (Tijdstip van functieherstel) - nodig
- stm_sap_storeinddatum (Datum einde storing) - nodig
- stm_sap_storeindtijd (Tijdstip einde storing) - nodig
- stm_fh_duur (Duur van functiehersteltijd) - nodig

Over equipment:
- stm_equipm_nr_mld (equipment)
- stm_equipm_omschr_mld (equipment omschrijving)

Over locatie
- stm_km_van_mld (locatie van)
- stm_km_tot_mld (locatie tot)
- stm_vl_post (VL post) - nodig
- stm_contractgeb_mld(locatie melding) - nodig
- stm_pplg_van (PPLG van locatie)
- stm_pplg_naar (PPLG tot locatie)
- stm_dstrglp_van (Dienstregelpunt van locatie)
- stm_dstrglp_naar (Dienstregelpunt tot locatie)

Over prognose:
- stm_progfh_in_datum (Datum prognose)
- stm_progfh_in_tijd (Tijdstip FHT prognose)
- stm_progfh_in_invoer_dat (Invoer datum prognose)
- stm_progfh_in_invoer_tijd (Invoer tijdstip FHT prognose)
- stm_progfh_in_duur (Duur FHT)



In [3]:
#deze functie verwijdert alle rijen waar geen value op de gegeven kolommen zitten
def drop_rows_with_null(lst, df):
    dfisnull = df.isnull()[lst]
    matches = dfisnull[(dfisnull).any(axis=1)]
    return df.drop(matches.index)

In [4]:
# Verwijder de rijen die geen value bevatten in de benodigde kolommen
df_filtered = drop_rows_with_null(['stm_geo_mld', 'stm_sap_melddatum', 'stm_oorz_code', 'stm_sap_meldtijd',
                          'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd',
                          'stm_fh_duur', 'stm_vl_post', 'stm_contractgeb_mld'], df)
print(len(df))
print(len(df_filtered))
df_filtered

89852
13391


,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_sap_meldtekst,stm_geo_mld,stm_equipm_nr_mld,stm_equipm_soort_mld,...,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_oorz_tekst_kort,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar
12,125,50053438,62.0,AH,15/01/2006 23:08:00,Logboeknr RBV : 62 Tijd: 2308 VL-Post: ...,Logboeknr RBV : 62 Tijd: 2308 VL-P,514.0,10201441.0,WISSEL,...,15/01/2006,23:20:00,12.0,15/01/2006,23:20:00,NaN,NaN,NaN,Nm,NaN
34,358,50053800,2.0,AMF,05/02/2006 00:00:00,Logboeknr RBV : 2 Tijd: 0000 VL-Post: ...,Logboeknr RBV : 2 Tijd: 0000 VL-P,530.0,10247795.0,OVERWEG,...,05/02/2006,00:27:00,27.0,05/02/2006,00:27:00,vermoedelijk vandalisme.,NaN,NaN,Dld,NaN
37,388,50053842,218.0,AMR,06/02/2006 23:00:00,Logboeknr RBV : 218 Tijd: 2300 VL-Post: ...,Logboeknr RBV : 218 Tijd: 2300 VL-P,84.0,10264905.0,WISSEL,...,07/02/2006,00:00:00,60.0,07/02/2006,00:00:00,Grendelwissel te Haarlem Vam gestoord.,NaN,NaN,HLMVA,SPBR
38,396,50053854,351.0,EHV,07/02/2006 14:35:00,Logboeknr RBV : 351 Tijd: 1435 VL-Post: ...,Logboeknr RBV : 351 Tijd: 1435 VL-P,123.0,10646536.0,SYSTBOVENL,...,07/02/2006,14:45:00,10.0,08/02/2006,12:30:00,Geen SMC-melding.Geen onderzoek geweest.,NaN,NaN,Tb,NaN
44,474,50053973,10.0,KFH,17/02/2006 02:20:00,Logboeknr RBV : 10 Tijd: 0220 VL-Post: ...,Logboeknr RBV : 10 Tijd: 0220 VL-P,950.0,10524639.0,DETGRSSSL,...,17/02/2006,02:21:00,1.0,17/02/2006,02:21:00,instelling,NaN,NaN,Whz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89784,908062,98000491,2332.0,UT,10/09/2006 08:58:00,Hmla wsl 1011 LL GC,Hmla wsl 1011 LL GC,100.0,10689074.0,WISSEL,...,10/09/2006,09:24:00,25.0,10/09/2006,09:54:00,NaN,HMLA,NaN,HMLA,NaN
89802,908236,99003005,974425.0,RTD,21/10/2010 19:42:00,"Dtz-Dt : Km 70.3-69.8, rijplaten waaien omhoo...","Dtz-Dt : Km 70.3-69.8, rijplaten waaien",112.0,10107602.0,BAANLICHAA,...,21/10/2010,20:59:00,77.0,21/10/2010,21:00:00,NaN,SDM,NaN,SDM,DTZ
89828,908414,99003167,1994752.0,RTD,09/05/2013 20:33:00,Brd : [2+] Brandmelding Station 2 sprinklerpo...,Brd : [2+] Brandmelding Station 2 sprink,116.0,10590754.0,TUNCONSTR,...,09/05/2013,21:11:00,37.0,09/05/2013,21:30:00,NaN,KFHAZ,NaN,KFHAZ,KFHAZ
89838,908489,99003196,2410816.0,GVC,22/02/2014 19:25:00,Ledn : Ovw 30.4 gestoord.,Ledn : Ovw 30.4 gestoord.,103.0,10020957.0,OVERWEG,...,24/02/2014,14:55:00,2610.0,24/02/2014,15:24:00,Status gewijzigd van Concept naar Compl,NaN,GWT,LD,00/00/0000


In [5]:
#uitgeschreven lijsten zodat ik ze niet steeds hoef over te typen

# melding
df[['stm_sap_meldtekst', 'stm_geo_mld', 'stm_oorz_code', 'stm_prioriteit']]
# tijd
df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd']]
df[['stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']]
# equipment
df[['stm_equipm_omschr_mld', 'stm_equipm_nr_mld']]
# locatie
df[['stm_km_van_mld', 'stm_km_tot_mld', 'stm_vl_post', 'stm_contractgeb_mld', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar']]
# prognose
df[['stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur']]


,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur
0,NaN,00:00:00,NaN,00:00:00,99999999.0
1,NaN,00:00:00,NaN,00:00:00,99999999.0
2,NaN,00:00:00,NaN,00:00:00,99999999.0
3,NaN,00:00:00,NaN,00:00:00,99999999.0
4,NaN,00:00:00,NaN,00:00:00,99999999.0
...,...,...,...,...,...
89847,13/05/2013,02:30:00,13/05/2013,02:40:36,145
89848,10/05/2013,20:15:00,10/05/2013,20:15:37,31
89849,10/05/2013,23:45:00,10/05/2013,23:45:27,77
89850,NaN,00:00:00,NaN,00:00:00,99999999


# Baseline
Alle problemen worden in 3 uur opgelost.

In [6]:
begin_eind_tijd_dirty = df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']].dropna()
begin_eind_tijd_dirty # begin en eindtijd in dataframe gezet.

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd
0,02/01/2006,22:30:00,02/01/2006,22:36:00
1,02/01/2006,20:00:00,02/01/2006,20:00:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00
3,04/01/2006,11:40:00,04/01/2006,11:45:00
4,06/01/2006,10:30:00,06/01/2006,10:30:00
...,...,...,...,...
89845,10/05/2013,13:21:00,10/05/2013,13:27:00
89846,10/05/2013,13:21:00,10/05/2013,13:27:00
89847,10/05/2013,15:11:00,13/05/2013,02:30:00
89848,10/05/2013,19:41:00,10/05/2013,20:15:00


In [7]:
begin_eind_tijd_clean = begin_eind_tijd_dirty.loc[(begin_eind_tijd_dirty['stm_sap_meldtijd'] != '::') & (begin_eind_tijd_dirty['stm_sap_storeindtijd'] != '::')]

In [8]:
stm_sap_melddatum_tijd_object = begin_eind_tijd_clean['stm_sap_melddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_meldtijd']
stm_sap_melddatum_tijd = pd.DataFrame(stm_sap_melddatum_tijd_object,
                   columns=['Begin_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [9]:
stm_sap_einddatum_tijd_object = begin_eind_tijd_clean['stm_sap_storeinddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_storeindtijd']
stm_sap_einddatum_tijd = pd.DataFrame(stm_sap_einddatum_tijd_object,
                   columns=['Eind_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [10]:
result_begin_eind_tijd = pd.concat([begin_eind_tijd_clean, stm_sap_melddatum_tijd, stm_sap_einddatum_tijd],axis=1).dropna()
result_begin_eind_tijd # de samengevoegde waarde in de dataframe gezet

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
0,02/01/2006,22:30:00,02/01/2006,22:36:00,02/01/2006 22:30:00,02/01/2006 22:36:00
1,02/01/2006,20:00:00,02/01/2006,20:00:00,02/01/2006 20:00:00,02/01/2006 20:00:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00,03/01/2006 13:33:00,03/01/2006 13:34:00
3,04/01/2006,11:40:00,04/01/2006,11:45:00,04/01/2006 11:40:00,04/01/2006 11:45:00
4,06/01/2006,10:30:00,06/01/2006,10:30:00,06/01/2006 10:30:00,06/01/2006 10:30:00
...,...,...,...,...,...,...
89845,10/05/2013,13:21:00,10/05/2013,13:27:00,10/05/2013 13:21:00,10/05/2013 13:27:00
89846,10/05/2013,13:21:00,10/05/2013,13:27:00,10/05/2013 13:21:00,10/05/2013 13:27:00
89847,10/05/2013,15:11:00,13/05/2013,02:30:00,10/05/2013 15:11:00,13/05/2013 02:30:00
89848,10/05/2013,19:41:00,10/05/2013,20:15:00,10/05/2013 19:41:00,10/05/2013 20:15:00


In [46]:
result_begin_eind_tijd['Begin_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Begin_tijd_en_datum'].dt.strftime('%#d/%#m/%Y %H:%M:%S'))
result_begin_eind_tijd['Eind_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Eind_tijd_en_datum'].dt.strftime('%#d/%#m/%Y %H:%M:%S'))

In [47]:
fix_duur = result_begin_eind_tijd['Eind_tijd_en_datum'] - result_begin_eind_tijd['Begin_tijd_en_datum']

In [48]:
Duur_fix = pd.DataFrame(fix_duur,
                   columns=['Duur fix'])

In [49]:
Final_begin_eind_tijd = pd.concat([result_begin_eind_tijd, Duur_fix],axis=1).dropna()
Final_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
0,02/01/2006,22:30:00,02/01/2006,22:36:00,2006-01-02 22:30:00,2006-01-02 22:36:00,0 days 00:06:00
1,02/01/2006,20:00:00,02/01/2006,20:00:00,2006-01-02 20:00:00,2006-01-02 20:00:00,0 days 00:00:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00,2006-01-03 13:33:00,2006-01-03 13:34:00,0 days 00:01:00
3,04/01/2006,11:40:00,04/01/2006,11:45:00,2006-01-04 11:40:00,2006-01-04 11:45:00,0 days 00:05:00
4,06/01/2006,10:30:00,06/01/2006,10:30:00,2006-01-06 10:30:00,2006-01-06 10:30:00,0 days 00:00:00
...,...,...,...,...,...,...,...
89845,10/05/2013,13:21:00,10/05/2013,13:27:00,2013-05-10 13:21:00,2013-05-10 13:27:00,0 days 00:06:00
89846,10/05/2013,13:21:00,10/05/2013,13:27:00,2013-05-10 13:21:00,2013-05-10 13:27:00,0 days 00:06:00
89847,10/05/2013,15:11:00,13/05/2013,02:30:00,2013-05-10 15:11:00,2013-05-13 02:30:00,2 days 11:19:00
89848,10/05/2013,19:41:00,10/05/2013,20:15:00,2013-05-10 19:41:00,2013-05-10 20:15:00,0 days 00:34:00


In [50]:
Final_begin_eind_tijd.loc[(Final_begin_eind_tijd['Duur fix'] < '0 days 03:00:00')]

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
0,02/01/2006,22:30:00,02/01/2006,22:36:00,2006-01-02 22:30:00,2006-01-02 22:36:00,0 days 00:06:00
1,02/01/2006,20:00:00,02/01/2006,20:00:00,2006-01-02 20:00:00,2006-01-02 20:00:00,0 days 00:00:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00,2006-01-03 13:33:00,2006-01-03 13:34:00,0 days 00:01:00
3,04/01/2006,11:40:00,04/01/2006,11:45:00,2006-01-04 11:40:00,2006-01-04 11:45:00,0 days 00:05:00
4,06/01/2006,10:30:00,06/01/2006,10:30:00,2006-01-06 10:30:00,2006-01-06 10:30:00,0 days 00:00:00
...,...,...,...,...,...,...,...
89842,21/05/2016,13:17:56,21/05/2016,14:50:00,2016-05-21 13:17:56,2016-05-21 14:50:00,0 days 01:32:04
89845,10/05/2013,13:21:00,10/05/2013,13:27:00,2013-05-10 13:21:00,2013-05-10 13:27:00,0 days 00:06:00
89846,10/05/2013,13:21:00,10/05/2013,13:27:00,2013-05-10 13:21:00,2013-05-10 13:27:00,0 days 00:06:00
89848,10/05/2013,19:41:00,10/05/2013,20:15:00,2013-05-10 19:41:00,2013-05-10 20:15:00,0 days 00:34:00
